In [1]:
import pandas as pd
import numpy as np
import math
import time
from numpy.linalg import inv
from typing import Tuple
from IPython.display import Markdown, display
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split

In [2]:
# Sources for IWLS:
# https://github.com/jaeho3690/LogisiticRegression/blob/main/LogisticRegressionIRLS.ipynb

# Sources for GD & SGD:
# https://stackoverflow.com/questions/47795918/logistic-regression-gradient-descent
# https://medium.com/analytics-vidhya/gradient-descent-and-stochastic-gradient-descent-from-scratch-python-1cd93d4def49
# https://github.com/Darshansol9/GD-SGD_FromScratch_Python/blob/master/Code.ipynb

# Sources for ADAM:
# https://medium.com/analytics-vidhya/derivative-of-log-loss-function-for-logistic-regression-9b832f025c2d
# https://stackoverflow.com/questions/67080049/adam-optimization-for-gradient-descent-update-doesnt-seem-to-work-with-logistic?fbclid=IwAR3uW95_Entc1-esFrQtVhBKvIWE43781OW6OGIYWJAGLOa37o_z_tHpV0Q

# Source for cross-validation:
# https://github.com/jaeho3690/LogisiticRegression/blob/main/LogisticRegressionIRLS.ipynb

In [3]:
class LogisticRegression:
    """ Custom implementation of logistic regression algorithm"""
    
    def __sigmoid(self,x:float)->float:
        """ Activation function used to map any real value between 0 and 1 """
        return 1/(1+np.exp(-x))

    def loss_function(self,X:np.matrix,y:np.array,params:np.array,b:np.array=None)->float:
        """
        Computes the cost function for all the training samples 
        
        param: X - design matrix
        param: y - target vector comprising boolean value
        param: params - array of weights
        param: b - intercept (optional)
        """
        m,_ = X.shape
        if b:
            fx=self.__sigmoid(X.dot(params)+b)
        else:
            fx=self.__sigmoid(X.dot(params))
        cost=-np.sum(y * np.log(fx) + (1 - y)*np.log(1-fx))*(1/m)
        return cost
    
    def gradient_descent(self,X:np.matrix,y:np.array,params:np.array,iterations:int,alpha:float,min_delta:float,patience:int)->np.array:
        """
        Performs gradient descent optimization.
        
        Works assuming that weights vector contains
        intercept and the corresponding one column 
        has been added to the design matrix before 
        it is given to the method
        
        param: X - design matrix
        param: y - target vector comprising boolean value
        param: params - array of weights
        param: b - intercept (optional)
        param: iterations - number of iterations
        param: alpha - learning rate
        param: min_delta - minimum loss function change in the monitored quantity to qualify as improvement.
        param: patience - number of epochs with no improvement after which training will be stopped
        """
        # early stopping setup
        prev_loss,monitor = 0,0
        if min_delta and patience:
            X,X_val,y,y_val = train_test_split(X, y, test_size=0.1, random_state=42)

        for i in range(iterations):
            params = params+alpha*(X.T.dot(y-self.__sigmoid(X.dot(params))))
            # early stopping
            if min_delta and patience:
                loss = self.loss_function(X_val,y_val,params)
                cost_change = loss-prev_loss
                prev_loss = loss
                monitor=monitor+1 if (loss-prev_loss)<min_delta else 0
                if monitor==patience:
                    print(f'Fitting stopped after {monitor} iterations')
                    break
        return params
    
    def stochastic_gradient_descent(self,X:np.matrix,y:np.array,params:np.array,iterations:int,alpha:float,min_delta:float,patience:float,sample_size:int=1)->np.array:
        """
        Performs stochastic gradient descent optimization.
        
        Works assuming that weights vector contains
        intercept and the corresponding one column 
        has been added to the design matrix before 
        it is given to the method
        
        param: X - design matrix
        param: y - target vector comprising boolean value
        param: params - array of weights
        param: b - intercept (optional)
        param: iterations - number of iterations
        param: alpha - learning rate
        param: sample_size - batch size
        param: min_delta - minimum loss function change in the monitored quantity to qualify as improvement.
        param: patience - number of epochs with no improvement after which training will be stopped
        """
        # early stopping setup
        if min_delta and patience:
            prev_loss,monitor = 0,0
            X,X_val,y,y_val = train_test_split(X, y, test_size=0.1, random_state=42)
        
        # optimization setup
        assert sample_size <= X.shape[0]
        df_X = pd.DataFrame(X)
        df_y = pd.DataFrame(y)
        
        for i in range(iterations):
            n_samples = math.ceil(df_X.shape[0]/sample_size)
            shuffled = df_X.sample(frac=1)
            samples = np.array_split(shuffled, n_samples)
            for sample in samples:
                X_st = np.array(sample)
                y_st = np.array(y[sample.index])
                # y_st = np.expand_dims(y_st, axis=-1)
                params = params + alpha * (X_st.T.dot(y_st - self.__sigmoid(X_st.dot(params))))
            # early stopping
            if min_delta and patience:
                loss = self.loss_function(X_val,y_val,params)
                cost_change = loss-prev_loss
                prev_loss = loss
                monitor=monitor+1 if (loss-prev_loss)<min_delta else 0
                if monitor==patience:
                    print(f'Fitting stopped after {monitor} iterations')
                    break
        return params
    
    def irls(self,X:np.matrix,y:np.array,min_delta:float,patience:int,iterations:int=1000)->np.array:
        """
        Performs Iterative-Reweighted Least Squares optimization.
        
        Works assuming that weights vector contains
        intercept and the corresponding one column 
        has been added to the design matrix before 
        it is given to the method
        
        param: X - design matrix
        param: y - target vector comprising boolean value
        param: iterations - number of iterations
        param: min_delta - minimum loss function change in the monitored quantity to qualify as improvement.
        param: patience - number of epochs with no improvement after which training will be stopped
        """
        
        # early stopping setup
        if min_delta and patience:
            prev_loss,monitor = 0,0
            X,X_val,y,y_val = train_test_split(X, y, test_size=0.1, random_state=42)
        
        # optimization setup
        params = np.zeros((X.shape[1],1))

        for i in range(iterations):
            y_ = self.__sigmoid(np.matmul(X,params))
            R = np.diag(np.ravel(y_*(1-y_)))
            grad = np.matmul(X.T,(y_-y))
            hessian = np.matmul(np.matmul(X.T,R),X)+0.001*np.eye(X.shape[1])
            params -= np.matmul(np.linalg.inv(hessian),grad)
            
            # early stopping
            if min_delta and patience:
                loss = self.loss_function(X=X_val,y=y_val,params=params)
                cost_change = loss-prev_loss
                prev_loss = loss
                monitor=monitor+1 if (loss-prev_loss)<min_delta else 0
                if monitor==patience:
                    print(f'Fitting stopped after {monitor} iterations')
                    break
            
        return params
    
    def adam(self,X:np.matrix,y:np.array,b1:float,b2:float,iterations:int,alpha:float,eps:float,min_delta:float,patience:int)->np.array:
        """
        Performs stochastic gradient descent optimization.
        
        Works assuming that weights vector does not 
        contain intercept - it is a separate variable (named b)
        and the design matrix does not include additional 
        one column
        
        param: X - design matrix
        param: y - target vector comprising boolean value
        param: iterations - number of iterations
        param: alpha - learning rate
        param: sample_size - batch size
        param: b - intercept (optional)
        param: b1, b2 - initial decay rates used when estimating 
            the first and second moments of the gradient
        param: min_delta - minimum loss function change in the monitored quantity to qualify as improvement.
        param: patience - number of epochs with no improvement after which training will be stopped
        """
        
        # early stopping setup
        if min_delta and patience:
            prev_loss,monitor = 0,0
            X,X_val,y,y_val = train_test_split(X, y, test_size=0.1, random_state=42)
        
        # optimization setup
        m,n = X.shape
        W,b = np.random.randn(n,1),np.random.randn(1)
        VW,Vb = np.zeros((n,1)),np.zeros(1)
        SW,Sb = np.zeros((n,1)),np.zeros(1)
        prev_loss,monitor = 0,0
        y = y.reshape(len(y),1)
        
        for i in range(iterations):
            # sigmoid
            A = self.__sigmoid((X.dot(W)+b))
    
            # binary classification cost
            j = self.loss_function(X=X,y=y,params=W,b=b)  # (-y*np.log(A)-(1-y)*np.log(1-A)).sum()*(1/m)

            # derivative respect to j
            dA = (A-y)/(A*(1-A))
            dZ = A-y

            dW = X.transpose().dot(dZ)
            db = dZ.sum()
            
            # momentum
            VW = b1*VW + (1-b1)*dW
            Vb = b1*Vb + (1-b1)*db
            
            # rmsprop
            SW = b2*SW + (1-b2)*dW**2
            Sb = b2*Sb + (1-b2)*db**2
            
            # update weight
            W -= alpha*VW/(np.sqrt(SW)+eps)
            b -= alpha*Vb/(np.sqrt(Sb)+eps)
            
            # early stopping
            if min_delta and patience:
                loss = self.loss_function(X=X_val,y=y_val,params=W,b=b)
                cost_change = loss-prev_loss
                prev_loss = loss
                monitor=monitor+1 if (loss-prev_loss)<min_delta else 0
                if monitor==patience:
                    print(f'Fitting stopped after {monitor} iterations')
                    break
            
        return W,b
    
    
    def fit(self,X:np.matrix,y:np.array,**kwds)->np.array:
        """
        Fits the model according to the given training data.
        
        param: X - design matrix
        param: y - target vector comprising boolean value
        """
        
        if len(kwds.keys()) == 4:
            display(Markdown('### Gradient Descent\n'))
            # parameters setting
            # X = np.c_[np.ones((X.shape[0], 1)), X]
            params = np.random.randn(X.shape[1])
            params = params[:,np.newaxis]
            iterations = kwds["iterations"]
            learning_rate = kwds["alpha"]
            min_delta = kwds["min_delta"]
            patience = kwds["patience"]
            # optimization
            initial_cost = self.loss_function(X,y,params)
            start = time.time()
            params_optimal  = self.gradient_descent(X,y,params,iterations,learning_rate,min_delta,patience)
            end = time.time()
            final_cost = self.loss_function(X, y, params_optimal)
            
        elif len(kwds.keys()) == 5:
            display(Markdown('### Stochastic Gradient Descent\n'))
            # parameters setting
            # X = np.c_[np.ones((X.shape[0], 1)), X]
            params = np.random.randn(X.shape[1])
            params = params[:,np.newaxis]
            iterations = kwds["iterations"]
            learning_rate = kwds["alpha"]
            sample_size = kwds["sample_size"]
            min_delta = kwds["min_delta"]
            patience = kwds["patience"]
            # optimization
            initial_cost = self.loss_function(X=X,y=y,params=params)
            start = time.time()
            params_optimal  = self.stochastic_gradient_descent(X,y,params,iterations,learning_rate,min_delta,patience,sample_size)
            end = time.time()
            final_cost = self.loss_function(X=X,y=y,params=params_optimal)
            
        elif len(kwds.keys()) == 7:
            display(Markdown('### ADAM\n'))
            # parameters setting
            params=np.random.randn(X.shape[1],1)
            b1=kwds["b1"]
            b2=kwds["b2"]
            iterations=kwds["iterations"]
            learning_rate=kwds["alpha"]
            eps=kwds["epsilon"]
            min_delta = kwds["min_delta"]
            patience = kwds["patience"]
            # optimization
            m,n = X.shape
            W,b = np.random.randn(n,1),np.random.randn(1)
            initial_cost=self.loss_function(X=X,y=y,params=W,b=b)
            start=time.time()
            W,b=self.adam(X,y,b1=b1,b2=b2,iterations=iterations,alpha=learning_rate,eps=eps,min_delta=min_delta,patience=patience)
            end=time.time()
            final_cost=self.loss_function(X=X,y=y,params=W,b=b)
            params_optimal=(W,b)
        else:
            display(Markdown('### Iterative-Reweighted Least Squares\n'))
            
            # X = np.c_[np.ones((X.shape[0], 1)), X]
            # parameters setting
            try:
                min_delta = kwds["min_delta"]
                patience = kwds["patience"]
            except Exception as e:
                min_delta,patience = None,None
                
            try:
                iterations=kwds["iterations"]
                start = time.time()
                # optimization
                params_optimal  = self.irls(X,y,min_delta,patience,iterations=iterations)
                end = time.time()
            except Exception as e:
                start = time.time()
                # optimization
                params_optimal  = self.irls(X,y,min_delta,patience)
                end = time.time()
            params = np.zeros((X.shape[1],1))
            initial_cost = self.loss_function(X=X,y=y,params=params)
            final_cost = self.loss_function(X=X,y=y,params=params_optimal)
            
        print(f'Time eclapsed for fitting: {end-start} secs')
        print('Initial cost ',initial_cost)
        print('Final cost ',final_cost)
        print('\n\n')
        return params_optimal
    
    def predict(self,X:np.matrix,params:np.array,b:np.array=None,threshold=.5)->Tuple[np.array,np.array]:
        """
        Predicts probability estimates and labels for samples in X.
        
        param: X - design matrix
        param: params - array of weights
        param: b - intercept (optional)
        param: threshold - probability threshold. If the probability estimate
            returned for a given observation exceeds its value, the observation
            is assigned to the positive class
        """
        if b:
            prob_pred=self.__sigmoid(X.dot(params)+b)
        else:
            # X = np.c_[np.ones((X.shape[0], 1)), X]
            prob_pred=self.__sigmoid(X.dot(params))
        return prob_pred, (prob_pred > threshold).astype(int)

## Test

In [4]:
credit_train_x = pd.read_csv("../datasets/preprocessed/credit_train_x.csv")
credit_train_y = pd.read_csv("../datasets/preprocessed/credit_train_y.csv")

credit_train_x = np.array(credit_train_x)
credit_train_y = np.array(credit_train_y)

X, y = credit_train_x, credit_train_y

In [5]:
# Gradient descent
iterations = 1000
learning_rate = 2e-5
min_delta=0.01
patience = 3

gd_kwds={
    "iterations": iterations,
    "alpha": learning_rate,
    "min_delta":min_delta,
    "patience":patience
}

lr = LogisticRegression()
X_ = np.c_[np.ones((X.shape[0], 1)), X]
W=lr.fit(X_,y,**gd_kwds)
prob_pred,y_pred=lr.predict(X_, W)

roc_auc_score(y, prob_pred)

### Gradient Descent


Fitting stopped after 3 iterations
Time eclapsed for fitting: 0.002100229263305664 secs
Initial cost  1.4666213781440376
Final cost  1.4591837683677962





0.5658835978835979

In [6]:
# Stochastic gradient descent
iterations = 2000
learning_rate = 2e-5
sample_size = 1
min_delta=0.01
patience = 3

sgd_kwds={
    "iterations": iterations,
    "alpha": learning_rate,
    "sample_size": sample_size,
    "min_delta":min_delta,
    "patience":patience
}

lr = LogisticRegression()
X_ = np.c_[np.ones((X.shape[0], 1)), X]
W=lr.fit(X_,y,**sgd_kwds)
prob_pred,y_pred=lr.predict(X_, W)

roc_auc_score(y, prob_pred)

### Stochastic Gradient Descent


Fitting stopped after 3 iterations
Time eclapsed for fitting: 0.29991769790649414 secs
Initial cost  1.0967238213506627
Final cost  1.0883173114668894





0.5588994708994709

In [7]:
# ADAM
b1=0.9
b2=0.999
iterations=20000
alpha=2e-5
eps=1e-8
min_delta=0.01
patience = 3

adam_kwds={
    "iterations": iterations,
    "b1": b1,
    "b2": b2,
    "alpha": alpha,
    "epsilon": eps,
    "min_delta":min_delta,
    "patience":patience
}
lr = LogisticRegression()
W,b=lr.fit(X,y,**adam_kwds)
prob_pred,y_pred=lr.predict(X=X,params=W,b=b)

roc_auc_score(y, prob_pred)

### ADAM


Fitting stopped after 3 iterations
Time eclapsed for fitting: 0.0029866695404052734 secs
Initial cost  3.493748328120349
Final cost  1.8119368745687718





0.48915978835978835

In [8]:
# Iterative-Reweighted Least Squares
iterations=10
min_delta=0.01
patience = 3
lr = LogisticRegression()
kwds={
    "iterations": iterations,
    "min_delta":min_delta,
    "patience":patience
}

lr = LogisticRegression()
X_ = np.c_[np.ones((X.shape[0], 1)), X]
W=lr.fit(X_,y,**kwds)
prob_pred,y_pred=lr.predict(X_, W)

roc_auc_score(y, prob_pred)

### Iterative-Reweighted Least Squares


Fitting stopped after 3 iterations
Time eclapsed for fitting: 0.0211794376373291 secs
Initial cost  0.6931471805599453
Final cost  0.44193395841509664





0.8388402116402116

In [9]:
# Iterative-Reweighted Least Squares
lr = LogisticRegression()
X_ = np.c_[np.ones((X.shape[0], 1)), X]
W=lr.fit(X_,y)
prob_pred,y_pred=lr.predict(X_, W)

roc_auc_score(y, prob_pred)

### Iterative-Reweighted Least Squares


Time eclapsed for fitting: 3.4219419956207275 secs
Initial cost  0.6931471805599453
Final cost  0.4397883680889956





0.8394666666666666